<a href="https://colab.research.google.com/github/Whereamiactually/lyceumcompling11/blob/main/Chatbots_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Чат-боты!

Сегодня у нас в планах посмотреть на простого бота для телеграма. Вот [тут](https://core.telegram.org/bots/api) лежит документация, где можно найти, какие запросы нужно отправлять к API.

Первый шаги:

1. Откройте приложение Telegram и найдите бота @BotFather.
2. Напишите боту /newbot и следуйте инструкциям, чтобы создать своего бота.
3. Сохраните полученный токен доступа к вашему боту (вида 6814073858:AAEGGyn...).

Profit! Бот рожден и теперь к нему можно обращаться, например, через браузер.

Чтобы проверить, что бот действительно работает, можно отправить ему вот такой запрос: `https://api.telegram.org/bot*your-bot-token*/getme` (обратите внимание, что на месте \*your-bot-token\*, вам нужно вставить токен своего бота).

А с помощью вот этого запроса `https://api.telegram.org/bot*your-bot-token*/getUpdates` можно посмотреть на все сообщения, которые посылались вашему боту за все время его работы. Пока у вас там пусто.

Если вы напишете самому боту, то он вам, конечно, ничего не ответит, потому что пока внутри него не прописаны никакие команды. Займёмся этим!

Установим библиотеку для работы с ботом.

Существует довольно много оберток для работы с ботами для телеграма: [python-telegram-bot](https://github.com/python-telegram-bot/python-telegram-bot), [telepot](https://github.com/nickoala/telepot), [pyTelegramBotAPI](https://github.com/eternnoir/pyTelegramBotAPI). Мы будем использовать последнюю.

In [1]:
!pip install pyTelegramBotAPI

In [1]:
import telebot # импортируем модуль

Нельзя палить токен вашего бота! Потому что иначе ботом сможет воспользоваться любой и натворить в нём гадостей.

Чтобы этого не допустить, надо создать отдельный файл `conf.py` и записать в него cтроку `TOKEN = "ваш токен"`.

Это нужно для того, чтобы не выкладывать в репозиторий свои логины, пароли и токены доступа. Чтобы ничего не стирать перед выкладыванием в репозиторий, лучше всего выносить секретные данные в отдельный файл и сделать так, чтобы этот файл игнорировался Git'ом:

* создайте в корне файл с названием `.gitignore`,
* напишите в этом файле `conf.py`,
* после этого файл будет у вас на компьютере, но никогда не запушится в репозиторий (чуть подробнее про это можно почитать [тут](https://learn.microsoft.com/ru-ru/azure/devops/repos/git/ignore-files?view=azure-devops&tabs=visual-studio-2022)).

В основной же файл с программой этот файл можно будет просто импортировать строчкой `import conf`. Тогда переменная `TOKEN` будет доступна внутри основной программы как `conf.TOKEN`.



In [2]:
import conf # импортируем файл, в котором лежит наш токен

In [3]:
bot = telebot.TeleBot(conf.TOKEN) # создаем эксземпляр бота

Создадим бота, который говорит, сколько букв в запросе пользователя.

In [9]:
# этот обработчик запускает функцию send_welcome, когда пользователь отправляет команды /start или /help
@bot.message_handler(commands = ['start', 'help']) # прописываем, на какие команды надо реагировать
def send_welcome(message): # определяем функцию, которая применяется при этих командах
    bot.send_message(message.chat.id, "Здравствуйте! Это бот, который считает длину вашего сообщения.")


# этот обработчик реагирует на любое сообщение
@bot.message_handler(func = lambda m: True)
def send_len(message):
    bot.send_message(message.chat.id, 'В вашем сообщении {} символов.'.format(len(message.text)))

Внутри декоратора `@bot.message_handler()` могут находиться фильтры, которые описывают, на какие сообщения реагирует данная функция.

Фильтры пишутся так: сначала название фильтра, затем через знак равенства его значение.

Бывают фильтры четырех типов:

* **content_types** - значением является массив строк, задающих тип контента - текст, аудио, файл, стикер и т.д. (по умолчанию ['text']);
* **regexp** - значением является регулярное выражение в виде строки;
* **commands** - значением является массив строк, задающих команды (без знака "/");
* **func** - значением является какая-то функция.

Если боту приходит сообщение, которое подходит под несколько фильтров, т.е. несколько разных функций, в этом случае запускается функция, которая в вашем коде написана раньше других.

В переменной `message` лежат все данные о конкретном запросе пользователя. В коде выше мы использовали атрибут `.chat.id`, который возвращает уникальный номер пользователя, написавшего сообщение, и `.text`, который возвращает текст сообщения.

Это все здорово, конечно, но если мы попытаемся запустить этот код и начать писать нашему боту, то ничего не произойдет... Почему? Потому что наш бот не получает наши сообщения и думает, что полковнику никто не пишет. :(

Исправим это следующей строчкой кода.

Этот код просит бота бесконечно опрашивать сервера телеграма на предмет новых сообщений (как-то так: "Мне что-нибудь пришло? А сейчас пришло что-нибудь? А сейчас? Пришло? Пришло? А сейчас написали что-нибудь мне? Мы уже приехали?"). Параметр `none_stop = True` говорит, что бот должен стараться не прекращать работу при возникновении каких-либо ошибок.

In [10]:
if __name__ == '__main__':
    bot.polling(none_stop = True)

### Ещё один бот

Напишем бота, который в ответ на команду `/dog` получается картинку с случайной собачкой.

In [5]:
import requests # надо отдельно импортировать модули, которые будут использоваться в коде

In [ ]:
@bot.message_handler(commands = ['start', 'help'])
def send_welcome(message):
    bot.send_message(message.chat.id, "Здравствуйте! Это бот, который порадует вас картинкой щеночка.")

@bot.message_handler(commands = ['dog']) # прописываемы тут нужную нам команду
def bop(message):
    contents = requests.get('https://random.dog/woof.json').json()
    url = contents['url']
    bot.send_photo(chat_id = message.chat.id, photo = url) # прописываем специальную функцию, которая возвращает пользователю картинку

Вот [здесь](https://habr.com/ru/companies/macloud/articles/562700/) можно найти 17 прикольных сервисов с API, которые вы можете использовать как основу для вашего бота!

In [ ]:
if __name__ == '__main__':
    bot.polling(none_stop = True)

### Ещё два бот

Хотим завести попугая!

In [4]:
@bot.message_handler(commands = ['start', 'help'])
def help(message):
    user = message.chat.id
    bot.send_message(user, "Я попугай и я вас попугаю!")

@bot.message_handler(content_types = ['text']) # сработает на любое текстовое сообщение
def echo(message):
    text = message.text
    user = message.chat.id

    # отправляем картинку с попугаем
    bot.send_photo(user, "https://jakosha.ru/wp-content/uploads/2015/06/intelekt_popugaya.jpg")

    # отправляем сообщение тому же пользователю с тем же текстом
    if text == "Я дурак.":
      bot.send_message(user, "Ты дурак.")
    else:
      bot.send_message(user, text)

bot.polling(none_stop = True)

### Ещё три бот

Попробуем применить простую штучку из тех, что вы прошли на этом курсе. Допустим, хотим создать бот, который выдает части речи всех слов нашего сообщения.

In [5]:
import nltk # вначале надо импортировать все нужные нам библиотеки и штуки

In [6]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
@bot.message_handler(commands = ['start', 'help'])
def send_welcome(message):
    bot.send_message(message.chat.id, "Здравствуйте! Это бот, который выдаст вам части слов из вашего сообщения.")

@bot.message_handler(func = lambda m: True) # пускай он реагирует на все сообщения
def bop(message):
    tokens = nltk.word_tokenize(message.text)
    tagged_tokens = nltk.pos_tag(tokens)
    bot.send_message(message.chat.id, tagged_tokens)

In [9]:
if __name__ == '__main__':
    bot.polling(none_stop = True)

### Другие варианты получения ботом сообщений

Вообще вариантов получения ботом сообщений от телеграма два:

1. Опрос (от англ. *polling*) сервера телеграма на наличие сообщений для бота.
2. "Почтовый ящик" с ip-адресом (*webhook* - можно перевести как "веб-ловушка"), на который приходят сообщения от сервера телеграма.

Вот [тут](https://habr.com/ru/company/ods/blog/462141/) можно подробно про это всё почитать.

Пока используем *polling*, но это не оптимальное решение, так как, если вы выложите ваше приложение на какой-нибудь веб-сервер и на сервере произойдет хоть какая-нибудь ошибка или дисконнект, бот перестанет работать.

Поэтому попозже, если будет время, мы посмотрим на то, как использовать вебхуки, но их необязательно будет использовать в вашем боте.

### Последний бот на сегодня

Как думаете, что он делает?

In [ ]:
import telebot
import conf

bot = telebot.TeleBot(conf.token)

notes = {}

@bot.message_handler(commands = ['remind'])
def remind(message):
    user_id = message.chat.id
    if user_id not in notes:
        bot.send_message(user_id, "Вы мне еще не писали.")
    else:
        bot.send_message(user_id, notes[user_id])

@bot.message_handler(content_types = ['text'])
def remember(message):
    user_id = message.chat.id
    notes[user_id] = message.text
    bot.send_message(user_id, "Я запомнил.")

bot.polling(none_stop = True)